In [2]:

# Define image loading and processing function
def process_image(image_path):
    try:
        with Image.open(image_path) as img:
            img = img.resize((224, 224))
            img = np.array(img.convert('RGB'), dtype=np.float32) / 255.0  # Normalize
            return img
    except FileNotFoundError:
        print(f"Warning: File {image_path} not found.")
        return None

# Create a generator to yield data
def data_generator(image_paths, labels):
    for path_tuple, label in zip(image_paths, labels):
        anchor_img = process_image(path_tuple[0])
        positive_img = process_image(path_tuple[1])
        negative_img = process_image(path_tuple[2])
        # Explicit check for None in images
        if anchor_img is None or positive_img is None or negative_img is None:
            continue
        yield (anchor_img, positive_img, negative_img), np.array([label], dtype=np.int32)


# Function to create dataset
# Function to create dataset
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def process_image(image_path):
    try:
        with Image.open(image_path) as img:
            img = img.resize((224, 224))
            img = np.array(img.convert('RGB'), dtype=np.float32) / 255.0  # Normalize
            img = datagen.random_transform(img)  # Apply data augmentation
            return img
    except FileNotFoundError:
        print(f"Warning: File {image_path} not found.")
        return None


# # Define image loading and processing function
# def process_image(image_path):
#     try:
#         with Image.open(image_path) as img:
#             img = img.resize((224, 224))
#             img = np.array(img.convert('RGB'), dtype=np.float32) / 255.0  # Normalize
#             return img
#     except FileNotFoundError:
#         print(f"Warning: File {image_path} not found.")
#         return None

# Create a generator to yield data
def data_generator(image_paths, labels):
    for path_tuple, label in zip(image_paths, labels):
        anchor_img = process_image(path_tuple[0])
        positive_img = process_image(path_tuple[1])
        negative_img = process_image(path_tuple[2])
        # Explicit check for None in images
        if anchor_img is None or positive_img is None or negative_img is None:
            continue
        yield (anchor_img, positive_img, negative_img), np.array([label], dtype=np.int32)


# Function to create dataset
# Function to create dataset
import numpy as np
import tensorflow as tf

def create_dataset(df, image_dir, batch_size):
    def generator():
        for _, row in df.iterrows():
            anchor = process_image(os.path.join(image_dir, row['anchor']))
            positive = process_image(os.path.join(image_dir, row['pos']))
            negative = process_image(os.path.join(image_dir, row['neg']))
            if anchor is not None and positive is not None and negative is not None:
                yield (anchor, positive, negative), 0  # Dummy label, not used

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            (tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
             tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
             tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32)),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )

    num_samples = len(df)
    steps_per_epoch = np.ceil(num_samples / batch_size).astype(int)  # Ensure it's an integer

    return dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE).repeat(), steps_per_epoch


In [3]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar

# Paths
csv_path = '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/triplets.csv'
image_dir = '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images'

# Load CSV
data = pd.read_csv(csv_path)

train = data.head(3000)
test = data.tail(1000)

In [4]:
train_dataset, steps_per_epoch = create_dataset(train, image_dir, batch_size=32)
val_dataset, validation_steps = create_dataset(test, image_dir, batch_size=32)


In [5]:
def triplet_loss(margin=1.0):
    def loss(y_true, y_pred):
        total_length = y_pred.shape[1] // 3
        anchor = y_pred[:, :total_length]
        positive = y_pred[:, total_length:2*total_length]
        negative = y_pred[:, 2*total_length:]

        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
        
        basic_loss = pos_dist - neg_dist + margin
        loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0))
        return loss
    
    return loss

In [6]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2
import tensorflow as tf


def build_siamese_model2(input_shape=(224, 224, 3)):
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')

    input_anchor = layers.Input(shape=input_shape, name='anchor')
    input_positive = layers.Input(shape=input_shape, name='positive')
    input_negative = layers.Input(shape=input_shape, name='negative')

    encoded_anchor = base_model(input_anchor)
    encoded_positive = base_model(input_positive)
    encoded_negative = base_model(input_negative)

    # Apply L2 regularization
    encoded_anchor = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01))(encoded_anchor)
    encoded_positive = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01))(encoded_positive)
    encoded_negative = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01))(encoded_negative)

    # Reshape the output to 3D tensor for LSTM layer
    encoded_anchor = layers.Reshape((1, -1))(encoded_anchor)
    encoded_positive = layers.Reshape((1, -1))(encoded_positive)
    encoded_negative = layers.Reshape((1, -1))(encoded_negative)

    # Add LSTM layers
    lstm_layer = layers.LSTM(256, return_sequences=False)
    encoded_anchor = lstm_layer(encoded_anchor)
    encoded_positive = lstm_layer(encoded_positive)
    encoded_negative = lstm_layer(encoded_negative)

    merged_output = layers.concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1)

    model = Model(inputs=[input_anchor, input_positive, input_negative], outputs=merged_output)
    model.compile(optimizer='adam', loss=triplet_loss(), run_eagerly=True)

    return model

# Instantiate the model
model2 = build_siamese_model2()

# Model summary
model2.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ anchor (InputLayer) │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positive            │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ negative            │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mobilenetv2_1.00_2… │ (None, 1280)      │  2,257,984 │ anchor[0][0],     │
│ (Functional)        │                   │            │ positive[0][0],   │
│                     │                   │            │ negative[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    163,968 │ mobilenetv2_1.00… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │    163,968 │ mobilenetv2_1.00… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 128)       │    163,968 │ mobilenetv2_1.00… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 128)    │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 1, 128)    │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 1, 128)    │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    394,240 │ reshape[0][0],    │
│                     │                   │            │ reshape_1[0][0],  │
│                     │                   │            │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 768)       │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ lstm[1][0],       │
│                     │                   │            │ lstm[2][0]        │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,144,128 (11.99 MB)

 Trainable params: 3,110,016 (11.86 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Setup callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
    ModelCheckpoint('best_model.keras',  # Path where the model will be saved
        monitor='val_loss',  # Monitor validation loss to determine the best model
        save_best_only=True,  # Save only the best model
        verbose=1  # Output messages
    )
]


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Setup callbacks with reduced patience
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1),
    ModelCheckpoint('best_model3.keras',  # Path where the model will be saved
        monitor='val_loss',  # Monitor validation loss to determine the best model
        save_best_only=True,  # Save only the best model
        verbose=1  # Output messages
    )
]


In [10]:
try:
    # Fit model
    history_mobile_net_v2_2 = model2.fit(
        train_dataset,
        epochs=50,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_dataset,
        validation_steps=validation_steps,
        callbacks=callbacks
    )
    
except KeyboardInterrupt:
    print("\nTraining Stopped!")

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 4.1275
Epoch 1: val_loss improved from inf to 0.21129, saving model to best_model3.keras
94/94 ━━━━━━━━━━━━━━━━━━━━ 299s 3s/step - loss: 4.1072 - val_loss: 0.2113 - learning_rate: 0.0010
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.1172
Epoch 2: val_loss improved from 0.21129 to 0.01231, saving model to best_model3.keras
94/94 ━━━━━━━━━━━━━━━━━━━━ 277s 3s/step - loss: 0.1167 - val_loss: 0.0123 - learning_rate: 0.0010
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.0088
Epoch 3: val_loss improved from 0.01231 to 0.00871, saving model to best_model3.keras
94/94 ━━━━━━━━━━━━━━━━━━━━ 277s 3s/step - loss: 0.0088 - val_loss: 0.0087 - learning_rate: 0.0010
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.0048
Epoch 4: val_loss improved from 0.00871 to 0.00152, saving model to best_model3.keras
94/94 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - loss: 0.0048 - val_loss: 0.0015 - learning_rate: 0.0010
Epoch 5/50
9

In [8]:
try:
    # Fit model
    history_mobile_net_v2_2 = model2.fit(
        train_dataset,
        epochs=50,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_dataset,
        validation_steps=validation_steps,
        callbacks=callbacks
    )
    
except KeyboardInterrupt:
    print("\nTraining Stopped!")

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 4.2258
Epoch 1: val_loss improved from inf to 0.29554, saving model to best_model2.keras
94/94 ━━━━━━━━━━━━━━━━━━━━ 308s 3s/step - loss: 4.2057 - val_loss: 0.2955 - learning_rate: 0.0010
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.1746
Epoch 2: val_loss improved from 0.29554 to 0.02231, saving model to best_model2.keras
94/94 ━━━━━━━━━━━━━━━━━━━━ 270s 3s/step - loss: 0.1739 - val_loss: 0.0223 - learning_rate: 0.0010
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.0150
Epoch 3: val_loss improved from 0.02231 to 0.00470, saving model to best_model2.keras
94/94 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - loss: 0.0149 - val_loss: 0.0047 - learning_rate: 0.0010
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 0.0038
Epoch 4: val_loss improved from 0.00470 to 0.00099, saving model to best_model2.keras
94/94 ━━━━━━━━━━━━━━━━━━━━ 267s 3s/step - loss: 0.0038 - val_loss: 9.9196e-04 - learning_rate: 0.0010
Epoch 5/

In [3]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import euclidean_distances as L2

def triplet_loss(margin=1.0):
    def loss(y_true, y_pred):
        total_length = y_pred.shape[1] // 3
        anchor = y_pred[:, :total_length]
        positive = y_pred[:, total_length:2*total_length]
        negative = y_pred[:, 2*total_length:]

        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
        
        basic_loss = pos_dist - neg_dist + margin
        loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0))
        return loss
    
    return loss

model_path = '/kaggle/input/facial-reg/best_model3.keras'  # Path to the saved model

# Load the trained model
model = load_model(model_path, custom_objects={'loss': triplet_loss()})

csv_path = '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/triplets.csv'
image_dir = '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images'
 # Path to the saved model

# Load CSV
data = pd.read_csv(csv_path)

# Split into train, test2, and test sets
train = data.head(3000)
test2 = data.iloc[3000:-10000]
test = data.tail(1000)


In [4]:
# Define image loading and processing function
def process_image(image_path):
    try:
        with Image.open(image_path) as img:
            img = img.resize((224, 224))
            img = np.array(img.convert('RGB'), dtype=np.float32) / 255.0  # Normalize
            return img
    except FileNotFoundError:
        print(f"Warning: File {image_path} not found.")
        return None

# Create a generator to yield test data
def test_data_generator(df, image_dir):
    for _, row in df.iterrows():
        anchor_img = process_image(os.path.join(image_dir, row['anchor']))
        positive_img = process_image(os.path.join(image_dir, row['pos']))
        negative_img = process_image(os.path.join(image_dir, row['neg']))
        if anchor_img is None or positive_img is None or negative_img is None:
            continue
        yield [anchor_img, positive_img, negative_img]

# Create test data arrays
anchor_imgs, positive_imgs, negative_imgs = [], [], []
for anchor_img, positive_img, negative_img in tqdm(test_data_generator(test, image_dir), total=len(test)):
    anchor_imgs.append(anchor_img)
    positive_imgs.append(positive_img)
    negative_imgs.append(negative_img)

anchor_imgs = np.array(anchor_imgs)
positive_imgs = np.array(positive_imgs)
negative_imgs = np.array(negative_imgs)



100%|██████████| 1000/1000 [00:23<00:00, 43.19it/s]


In [5]:
# Predict embeddings
y_pred = model.predict([anchor_imgs, positive_imgs, negative_imgs])

# Split the predictions into anchor, positive, and negative embeddings
total_length = y_pred.shape[1] // 3
anchor_emb = y_pred[:, :total_length]
positive_emb = y_pred[:, total_length:2*total_length]
negative_emb = y_pred[:, 2*total_length:]

# Calculate distances
distances = []
for i in range(len(test)):
    pos = L2(anchor_emb[i, :].reshape(1, -1), positive_emb[i, :].reshape(1, -1))[0][0]
    neg = L2(anchor_emb[i, :].reshape(1, -1), negative_emb[i, :].reshape(1, -1))[0][0]
    distances.append([pos, neg])

# Add distances to the test DataFrame
test[['pos_distance', 'neg_distance']] = pd.DataFrame(distances, index=test.index)

# Save the test set with new columns
test.to_csv('test_with_distances.csv', index=False)
print("Test set with distances saved to 'test_with_distances.csv'")

32/32 ━━━━━━━━━━━━━━━━━━━━ 15s 434ms/step
Test set with distances saved to 'test_with_distances.csv'


/tmp/ipykernel_34/692183653.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[['pos_distance', 'neg_distance']] = pd.DataFrame(distances, index=test.index)
/tmp/ipykernel_34/692183653.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[['pos_distance', 'neg_distance']] = pd.DataFrame(distances, index=test.index)


In [6]:
test

,anchor,id1,pos,id2,neg,id3,pos_distance,neg_distance
15332,189634.jpg,9481,197753.jpg,9481,009056.jpg,9601,0.166950,1.060237
15333,099772.jpg,9483,151488.jpg,9483,175385.jpg,3897,0.167215,1.077918
15334,138304.jpg,9483,016032.jpg,9483,163340.jpg,3483,0.167582,1.040132
15335,115943.jpg,9485,159473.jpg,9485,022629.jpg,9090,0.167341,1.083335
15336,081997.jpg,9485,127713.jpg,9485,043949.jpg,1595,0.169157,1.052204
...,...,...,...,...,...,...,...,...
16327,118516.jpg,10175,059929.jpg,10175,098802.jpg,5902,0.166897,1.164291
16328,056175.jpg,10176,152234.jpg,10176,130523.jpg,6261,0.166993,1.057978
16329,051029.jpg,10176,050089.jpg,10176,149607.jpg,718,0.166838,1.109627
16330,158603.jpg,10177,129750.jpg,10177,089851.jpg,6564,0.166895,1.087219


In [14]:
test

,anchor,id1,pos,id2,neg,id3,pos_distance,neg_distance
15332,189634.jpg,9481,197753.jpg,9481,009056.jpg,9601,0.166950,1.060237
15333,099772.jpg,9483,151488.jpg,9483,175385.jpg,3897,0.167215,1.077918
15334,138304.jpg,9483,016032.jpg,9483,163340.jpg,3483,0.167582,1.040132
15335,115943.jpg,9485,159473.jpg,9485,022629.jpg,9090,0.167341,1.083335
15336,081997.jpg,9485,127713.jpg,9485,043949.jpg,1595,0.169157,1.052204
...,...,...,...,...,...,...,...,...
16327,118516.jpg,10175,059929.jpg,10175,098802.jpg,5902,0.166897,1.164291
16328,056175.jpg,10176,152234.jpg,10176,130523.jpg,6261,0.166993,1.057978
16329,051029.jpg,10176,050089.jpg,10176,149607.jpg,718,0.166838,1.109627
16330,158603.jpg,10177,129750.jpg,10177,089851.jpg,6564,0.166895,1.087219


In [8]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import euclidean_distances as L2

# Paths
csv_path = '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/triplets.csv'
image_dir = '/kaggle/input/celeba-face-recognition-triplets/CelebA FR Triplets/CelebA FR Triplets/images'
model_path = '/kaggle/input/facial-reg/best_model3.keras'  # Path to the saved model

# Load CSV
data = pd.read_csv(csv_path)

# Split into train, test2, and test sets
train = data.head(3000)
test2 = data.iloc[3000:-10000]
test = data.tail(1000)


def triplet_loss(margin=1.0):
    def loss(y_true, y_pred):
        total_length = y_pred.shape[1] // 3
        anchor = y_pred[:, :total_length]
        positive = y_pred[:, total_length:2*total_length]
        negative = y_pred[:, 2*total_length:]

        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
        
        basic_loss = pos_dist - neg_dist + margin
        loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0))
        return loss
    
    return loss


# Load the trained model
model = load_model(model_path, custom_objects={'loss': triplet_loss()})

# Define image loading and processing function
def process_image(image_path):
    try:
        with Image.open(image_path) as img:
            img = img.resize((224, 224))
            img = np.array(img.convert('RGB'), dtype=np.float32) / 255.0  # Normalize
            return img
    except FileNotFoundError:
        print(f"Warning: File {image_path} not found.")
        return None

# Create a generator to yield test2 data
def test_data_generator(df, image_dir):
    for _, row in df.iterrows():
        anchor_img = process_image(os.path.join(image_dir, row['anchor']))
        positive_img = process_image(os.path.join(image_dir, row['pos']))
        negative_img = process_image(os.path.join(image_dir, row['neg']))
        if anchor_img is None or positive_img is None or negative_img is None:
            continue
        yield [anchor_img, positive_img, negative_img]

# Create test2 data arrays
anchor_imgs, positive_imgs, negative_imgs = [], [], []
for anchor_img, positive_img, negative_img in tqdm(test_data_generator(test2, image_dir), total=len(test2)):
    anchor_imgs.append(anchor_img)
    positive_imgs.append(positive_img)
    negative_imgs.append(negative_img)

anchor_imgs = np.array(anchor_imgs)
positive_imgs = np.array(positive_imgs)
negative_imgs = np.array(negative_imgs)



100%|██████████| 3332/3332 [01:14<00:00, 45.00it/s]


In [9]:
# Predict embeddings
y_pred = model.predict([anchor_imgs, positive_imgs, negative_imgs])

# Split the predictions into anchor, positive, and negative embeddings
total_length = y_pred.shape[1] // 3
anchor_emb = y_pred[:, :total_length]
positive_emb = y_pred[:, total_length:2*total_length]
negative_emb = y_pred[:, 2*total_length:]

# Calculate distances
distances = []
for i in range(len(test2)):
    pos = L2(anchor_emb[i, :].reshape(1, -1), positive_emb[i, :].reshape(1, -1))[0][0]
    neg = L2(anchor_emb[i, :].reshape(1, -1), negative_emb[i, :].reshape(1, -1))[0][0]
    distances.append([pos, neg])

# Add distances to the test2 DataFrame
test2[['pos_distance', 'neg_distance']] = pd.DataFrame(distances, index=test2.index)

# Save the test2 set with new columns
test2.to_csv('test2_with_distances.csv', index=False)
print("Test2 set with distances saved to 'test2_with_distances.csv'")


105/105 ━━━━━━━━━━━━━━━━━━━━ 45s 428ms/step
Test2 set with distances saved to 'test2_with_distances.csv'


/tmp/ipykernel_34/2073634733.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2[['pos_distance', 'neg_distance']] = pd.DataFrame(distances, index=test2.index)
/tmp/ipykernel_34/2073634733.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2[['pos_distance', 'neg_distance']] = pd.DataFrame(distances, index=test2.index)


In [10]:
test2

,anchor,id1,pos,id2,neg,id3,pos_distance,neg_distance
3000,070920.jpg,1743,086956.jpg,1743,045279.jpg,7022,0.168065,1.047356
3001,051497.jpg,1743,151161.jpg,1743,063649.jpg,6458,0.167116,1.036580
3002,082653.jpg,1744,106034.jpg,1744,029642.jpg,5923,0.167593,1.068511
3003,027321.jpg,1744,055318.jpg,1744,077988.jpg,3676,0.166785,1.065421
3004,016386.jpg,1745,002414.jpg,1745,111989.jpg,63,0.167220,1.079165
...,...,...,...,...,...,...,...,...
6327,009453.jpg,3839,019798.jpg,3839,004465.jpg,2339,0.167016,1.315190
6328,005448.jpg,3839,148472.jpg,3839,197483.jpg,6472,0.166770,1.044252
6329,084533.jpg,3840,011508.jpg,3840,049153.jpg,6996,0.167320,1.097032
6330,132631.jpg,3840,024084.jpg,3840,016896.jpg,4982,0.167360,1.102645


In [12]:
len(test2[(test2['pos_distance']<0.5) & (test2['neg_distance']>0.5)])/len(test2)

1.0

In [13]:
model.layers

[<InputLayer name=anchor, built=True>,
 <InputLayer name=positive, built=True>,
 <InputLayer name=negative, built=True>,
 <Functional name=mobilenetv2_1.00_224, built=True>,
 <Dense name=dense, built=True>,
 <Dense name=dense_1, built=True>,
 <Dense name=dense_2, built=True>,
 <Reshape name=reshape, built=True>,
 <Reshape name=reshape_1, built=True>,
 <Reshape name=reshape_2, built=True>,
 <LSTM name=lstm, built=True>,
 <Concatenate name=concatenate, built=True>]

In [15]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2
import tensorflow as tf

In [16]:
# Assuming 'model' is your already loaded full model
# Try to create the model directly with the known input shape if the layer reference fails
from tensorflow.keras.layers import Input

input_tensor = Input(shape=(224, 224, 3), name='anchor_input')
base_output = model.get_layer('mobilenetv2_1.00_224')(input_tensor)
embedding_output = model.get_layer('dense_2')(base_output)

# Create the new model
embedding_model = Model(inputs=input_tensor, outputs=embedding_output)

# Save the new model
embedding_model.save('Mobile_net_facial_embedder.h5')

# Verify by printing the model summary
embedding_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ anchor_input (InputLayer)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,421,952 (9.24 MB)

 Trainable params: 2,387,840 (9.11 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [19]:
embedder = load_model("/kaggle/working/Mobile_net_facial_embedder.h5")

In [21]:
!pip install streamlit mtcnn opencv-python-headless pycryptodome


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 78.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics.pairwise import euclidean_distances as L2, cosine_similarity as cs

In [22]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import euclidean_distances as L2
import math

# Initialize the face detector
detector = MTCNN()

# Function to compute Euclidean distance
def euclidean(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# Function to align the face
def align(x1, y1, x2, y2, img):
    x3, y3 = 0, 0
    adj, hyp = 0, euclidean(x1, y1, x2, y2)
    angle = 0
    
    if y1 > y2:
        x3, y3 = x1, y2
        adj = euclidean(x3, y3, x2, y2)
        angle = -math.degrees(math.acos(adj / hyp))
    else:
        x3, y3 = x2, y1
        adj = euclidean(x3, y3, x1, y1)
        angle = math.degrees(math.acos(adj / hyp))
        
    M = cv2.getRotationMatrix2D((img.shape[1] // 2, img.shape[0] // 2), angle, 1)
    out = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    return out

# Function to read and preprocess image
def read_image(directory, image_path):
    img = load_img(os.path.join(directory, image_path))
    img = img_to_array(img)
    faces = detector.detect_faces(img)
    x1, y1 = faces[0]['keypoints']['left_eye']
    x2, y2 = faces[0]['keypoints']['right_eye']
    img = align(x1, y1, x2, y2, img)
    faces = detector.detect_faces(img)
    x, y, w, h = faces[0]['box']
    img = img[y:y + h, x:x + w, :]
    img /= 255.0
    img = cv2.resize(img, (224, 224))
    return img

# # Load the embedding model
# model_path = "/kaggle/input/embedder/embedding_only_model.h5"
# embedder = load_model(model_path, compile=False)

# Directory containing the images
directory = '/kaggle/input/testing'

# Read and preprocess images
Abigail_img1 = read_image(directory, 'Abigail 1.jpg')
Abigail_img2 = read_image(directory, 'Abigail 2.jpg')
Kizito_img1 = read_image(directory, 'Kizito 1.jpg')
Kizito_img2 = read_image(directory, 'Kizito 2.jpg')

# Compute embeddings
emb_kizito1 = embedder.predict(np.expand_dims(Kizito_img1, axis=0))
emb_kizito2 = embedder.predict(np.expand_dims(Kizito_img2, axis=0))
emb_abigail1 = embedder.predict(np.expand_dims(Abigail_img1, axis=0))
emb_abigail2 = embedder.predict(np.expand_dims(Abigail_img2, axis=0))

# Compute Euclidean distance
distance_kizito = L2(emb_kizito1.reshape(1, -1), emb_kizito2.reshape(1, -1))
distance_abigail = L2(emb_abigail1.reshape(1, -1), emb_abigail2.reshape(1, -1))

print("Distance between Kizito images:", distance_kizito)
print("Distance between Abigail images:", distance_abigail)


2024-06-19 09:55:38.267462: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 373633: 4.05108, expected 3.53978
2024-06-19 09:55:38.267598: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 383666: 4.05108, expected 3.53978
2024-06-19 09:55:38.267699: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 393701: 4.05108, expected 3.53978
2024-06-19 09:55:38.267796: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 403734: 4.05108, expected 3.53978
2024-06-19 09:55:38.267894: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 413769: 4.05108, expected 3.53978
2024-06-19 09:55:38.267992: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 423802: 4.05108, expected 3.53978
2024-06-19 09:55:38.268045: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 433837: 4.05108, expected 3.53978
2024-06-19 09:55:38.

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 914ms/step


I0000 00:00:1718790938.751119     111 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-06-19 09:55:38.987441: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 10: 5.75492, expected 5.03988
2024-06-19 09:55:38.987489: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 11: 5.19202, expected 4.47698
2024-06-19 09:55:38.987499: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 12: 5.48391, expected 4.76887
2024-06-19 09:55:38.987508: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 24: 5.65626, expected 4.94123
2024-06-19 09:55:38.987515: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 27: 6.12535, expected 5.41031
2024-06-19 09:55:38.987523: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 28: 5.22754, expected 4.5125
2024-06-19 09:55:38.987531: E

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 629ms/step


2024-06-19 09:55:39.576997: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 46372: 4.91736, expected 4.2836
2024-06-19 09:55:39.577053: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 47047: 5.21202, expected 4.57826
2024-06-19 09:55:39.577064: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 47063: 5.20296, expected 4.56919
2024-06-19 09:55:39.577072: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 47181: 5.26029, expected 4.62652
2024-06-19 09:55:39.577082: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 47538: 4.98754, expected 4.35377
2024-06-19 09:55:39.577090: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 47602: 5.12607, expected 4.4923
2024-06-19 09:55:39.577098: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 47707: 5.25873, expected 4.62497
2024-06-19 09:55:39.577106: E

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


2024-06-19 09:56:00.671742: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 7.91061, expected 6.91811
2024-06-19 09:56:00.671794: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 8.74533, expected 7.75283
2024-06-19 09:56:00.671804: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 8.32064, expected 7.32814
2024-06-19 09:56:00.671812: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 7.074, expected 6.0815
2024-06-19 09:56:00.671820: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 7.74981, expected 6.75731
2024-06-19 09:56:00.671828: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 7: 7.40884, expected 6.41634
2024-06-19 09:56:00.671835: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 6.93266, expected 5.94016
2024-06-19 09:56:00.671843: E external/local_xla/xla/servi

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step


2024-06-19 09:56:01.197548: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 4.40306, expected 3.77572
2024-06-19 09:56:01.197600: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 24: 5.16637, expected 4.53902
2024-06-19 09:56:01.197617: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 166: 5.26059, expected 4.63324
2024-06-19 09:56:01.197632: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 375: 5.0737, expected 4.44635
2024-06-19 09:56:01.197644: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 402: 5.25195, expected 4.6246
2024-06-19 09:56:01.197655: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 420: 5.25032, expected 4.62297
2024-06-19 09:56:01.197665: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 427: 5.01308, expected 4.38573
2024-06-19 09:56:01.197676: E external/local_x

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Distance between Kizito images: [[0.01028967]]
Distance between Abigail images: [[0.02077716]]


In [25]:
distance_kizito = L2(emb_kizito1.reshape(1, -1), emb_kizito2.reshape(1, -1))
distance_abigail = L2(emb_abigail1.reshape(1, -1), emb_abigail2.reshape(1, -1))

print("Distance between Kizito images:", distance_kizito)
print("Distance between Abigail images:", distance_abigail)

Distance between Kizito images: [[0.01028967]]
Distance between Abigail images: [[0.02077716]]
